In [1]:
from pathlib import Path
import os
import sys

dirname = Path(os.path.abspath(''))
root = dirname.parent

sys.path.insert(0, str(root))

import torch

torch.__version__

'1.8.0'

## HyperParameters & Configurations

In [2]:
NORMALIZE_DATA = True
WINDOW_SIZE = 10
Y_DIM = 1
ENCODER_HIDDEN_STATES = 128
DECODER_HIDDEN_STATES = 128
BATCH_SIZE = 256
EPOCHS = 100
VALIDATION_RATIO = 0.2
DROPOUT = 0

## Read NASDAQ 100 raw data

In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

raw_data =  pd.read_csv('./nasdaq100_padding.csv').values

scale = StandardScaler().fit(raw_data)

if NORMALIZE_DATA:
    data = scale.transform(raw_data)
else:
    data = raw_data

In [4]:
from notebook.common import split_data
from torch.utils.data import TensorDataset, DataLoader


def to_tensor(array):
    return torch.from_numpy(array).float()
    

train_X, train_y, val_X, val_y = split_data(data, to_tensor, WINDOW_SIZE, Y_DIM, VALIDATION_RATIO)

print('train_X, train_y :', train_X.shape, train_y.shape)
print('  val_X,   val_y :', val_X.shape, val_y.shape)

train_X, train_y : torch.Size([32439, 10, 82]) torch.Size([32439, 1])
  val_X,   val_y : torch.Size([8103, 10, 82]) torch.Size([8103, 1])


In [16]:
from da_rnn.torch import DARNN, DEVICE
from poutyne import Model, EarlyStopping, ModelCheckpoint

darnn = DARNN(
    n=train_X.shape[2] - 1,
    T=WINDOW_SIZE,
    m=ENCODER_HIDDEN_STATES,
    p=DECODER_HIDDEN_STATES,
    y_dim=Y_DIM,
    dropout=DROPOUT
)

model = Model(
    darnn,
    'adam',
    'mse',
    device=DEVICE
)

save_to = dirname / 'checkpoint_torch.hdf5'

history = model.fit(
    train_X, train_y,
    validation_data=(val_X, val_y),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=5),
        ModelCheckpoint(
            str(save_to.absolute()),
            monitor='val_loss',
            verbose=1,
            save_best_only=True,
            keep_only_last_best=True
        )
    ]
)

Epoch:   1/100 Train steps: 127 Val steps: 32 53.81s loss: 0.227675 val_loss: 0.073213
Epoch 1: val_loss improved from inf to 0.07321, saving file to /Users/kael/Codes/ml/DA-RNN/notebook/checkpoint_torch.hdf5
Epoch:   2/100 Train steps: 127 Val steps: 32 53.86s loss: 0.004271 val_loss: 0.014231
Epoch 2: val_loss improved from 0.07321 to 0.01423, saving file to /Users/kael/Codes/ml/DA-RNN/notebook/checkpoint_torch.hdf5
Epoch:   3/100 Train steps: 127 Val steps: 32 54.45s loss: 0.003218 val_loss: 0.005791
Epoch 3: val_loss improved from 0.01423 to 0.00579, saving file to /Users/kael/Codes/ml/DA-RNN/notebook/checkpoint_torch.hdf5
Epoch:   4/100 Train steps: 127 Val steps: 32 54.72s loss: 0.002798 val_loss: 0.004166
Epoch 4: val_loss improved from 0.00579 to 0.00417, saving file to /Users/kael/Codes/ml/DA-RNN/notebook/checkpoint_torch.hdf5
Epoch:   5/100 Train steps: 127 Val steps: 32 54.58s loss: 0.002245 val_loss: 0.003145
Epoch 5: val_loss improved from 0.00417 to 0.00315, saving file t

In [ ]:
import matplotlib.pyplot as plt


def predict(model, X):
    with torch.no_grad():
        return model(X)
    

train_y_hat = predict(darnn, train_X)
plt.figure(figsize=(40,20))
plt.plot(
    scale.inverse_transform(train_y_hat.cpu().repeat(1,82).numpy())[:,-1],
    'k',
    scale.inverse_transform(train_y.repeat(1,82).numpy())[:,-1],
    'r'
)

val_y_hat = predict(darnn, val_X)
plt.figure(figsize=(40,20))
plt.plot(
    scale.inverse_transform(val_y_hat.cpu().repeat(1,82).numpy())[:,-1],
    'k',
    scale.inverse_transform(val_y.repeat(1,82).numpy())[:,-1],
    'r'
)